In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn import tree
from sklearn.model_selection import cross_val_score

# Datos

In [2]:
df=pd.read_excel('Bases de datos\dataset_escalado.xlsx')
df.head()

,ID_PDA,SN_PDA,FECHA_FAB,FECHA_TEST,FUNCION,RFID,USO,SN_BAT,ESTADO_BAT,GASTO_BAT,...,PWI_mAh_blue,FG_primer_plano_colum,PR_inicios,M_tiempo_bloqueo_activación_total,M_cambios_conectividad,APK_iniciosBluetooth,APK_iniciosGms,APK_iniciosLocation,APK_iniciosChrome,APK_iniciosGooglequicksearchbox
0,01042022EF500RANLGERKBM147,EF500RANLGERKBM147,2018-12,2022-04-01,2.0,0,0.0,355040034GBRL308273,75.0,64.0,...,-0.381557,-0.422714,-0.756545,-0.270046,0,-0.518817,-0.537426,-0.737793,-0.476592,-0.547770
1,01042022EF500RANLGERKBM388,EF500RANLGERKBM388,2018-08,2022-04-01,2.0,0,0.0,355040034GBRI603585,75.0,52.0,...,-0.192484,-0.421246,-0.716144,-0.385892,0,-0.518817,-0.552542,-0.737793,-0.476592,-0.488324
2,01042022EF500RANLGERLBO578,EF500RANLGERLBO578,2019-02,2022-04-01,2.0,0,0.0,355040058GBSBJ01338,75.0,35.0,...,-0.381557,-0.422058,-0.807046,-0.229903,0,-0.518817,-0.550457,-0.737793,-0.476592,-0.518047
3,01042022EF500RANLGERLBO599,EF500RANLGERLBO599,2018-12,2022-04-01,2.0,0,0.0,355040034GBRL307964,75.0,58.0,...,-0.311899,-0.225567,-0.844080,-0.312361,0,-0.518817,-0.547069,-0.737793,-0.476592,-0.587401
4,01042022EF500RANLGSQKBD779,EF500RANLGSQKBD779,2018-07,2022-04-01,2.0,0,0.0,355040034GBRGD04418,75.0,50.0,...,-0.381557,-0.010536,-0.770012,-0.385892,0,-0.294046,-0.329710,-0.737793,-0.476592,-0.567586


# Pre-procesado

In [3]:
df = df.drop(['ID_PDA','SN_PDA','FECHA_FAB','FECHA_TEST','SN_BAT'], axis=1)
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

clean_dataset(df)

,FUNCION,RFID,USO,ESTADO_BAT,GASTO_BAT,HORAS_TRABAJO,NUEVA_USADA,DC_baja,DC_alta,DC_pantalla_encendida,...,PWI_mAh_blue,FG_primer_plano_colum,PR_inicios,M_tiempo_bloqueo_activación_total,M_cambios_conectividad,APK_iniciosBluetooth,APK_iniciosGms,APK_iniciosLocation,APK_iniciosChrome,APK_iniciosGooglequicksearchbox
0,2.0,0.0,0.0,75.0,64.0,7.0,0.0,-0.534510,-0.565116,-1.109549,...,-0.381557,-0.422714,-0.756545,-0.270046,0.0,-0.518817,-0.537426,-0.737793,-0.476592,-0.547770
1,2.0,0.0,0.0,75.0,52.0,8.0,0.0,0.970963,0.980663,1.795812,...,-0.192484,-0.421246,-0.716144,-0.385892,0.0,-0.518817,-0.552542,-0.737793,-0.476592,-0.488324
2,2.0,0.0,0.0,75.0,35.0,5.0,0.0,-0.560467,-0.590879,-1.109549,...,-0.381557,-0.422058,-0.807046,-0.229903,0.0,-0.518817,-0.550457,-0.737793,-0.476592,-0.518047
3,2.0,0.0,0.0,75.0,58.0,8.0,0.0,-0.560467,-0.590879,-1.109549,...,-0.311899,-0.225567,-0.844080,-0.312361,0.0,-0.518817,-0.547069,-0.737793,-0.476592,-0.587401
4,2.0,0.0,0.0,75.0,50.0,8.0,0.0,0.763311,0.800322,0.402424,...,-0.381557,-0.010536,-0.770012,-0.385892,0.0,-0.294046,-0.329710,-0.737793,-0.476592,-0.567586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,6.0,0.0,0.0,75.0,15.0,8.0,1.0,-0.197077,-0.204434,-0.664851,...,-0.331801,-0.278447,0.263570,-0.332167,0.0,-0.518817,-0.056058,0.522549,-0.325204,-0.121737
104,6.0,0.0,0.0,75.0,18.0,8.0,1.0,0.348008,0.336589,-0.575912,...,-0.321850,-0.246109,0.186136,-0.320516,0.0,-0.518817,0.330444,0.942663,-0.249510,-0.151460
105,6.0,0.0,0.0,75.0,14.0,8.0,1.0,-0.223033,-0.230197,-0.694498,...,-0.331801,-0.290034,-0.157269,-0.333358,0.0,-0.518817,-0.286447,0.522549,-0.363051,-0.349615
106,6.0,0.0,0.0,75.0,13.0,8.0,1.0,-0.274946,-0.255960,-0.724144,...,-0.341752,-0.291453,0.559841,-0.337290,0.0,1.568344,-0.197836,1.362777,-0.060275,-0.111829


# Partición y balanceo clases

In [4]:
y = df['ESTADO_BAT'].values
X = df.drop(['ESTADO_BAT'], axis=1).values

sm = SMOTE(k_neighbors=2, sampling_strategy = "auto", random_state=100)
X, y = sm.fit_resample(X, y)

In [5]:
# Modelo
modelo = tree.DecisionTreeClassifier(max_depth = 5, splitter="random", random_state=10)
# Validación cruzada
f1_tree = cross_val_score(modelo, X, y, cv=10, scoring = 'f1_weighted').mean()
# Entrenamiento
fit = modelo.fit(X, y)

In [6]:
print(f'F1 Decision Tree: {f1_tree:.5f}')

F1 Decision Tree: 0.84475


# Guardar modelo

In [7]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(fit, open(filename, 'wb'))

In [8]:
############## Probar la predicción
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
# y = loaded_model.predict(X_test)
# print(result, y)